In [1]:
using Altro
using TrajectoryOptimization
using RobotDynamics
const RD = RobotDynamics
const TO = TrajectoryOptimization

using JuMP
using OSQP
using SparseArrays
using BenchmarkTools
using TimerOutputs
using Random
using Profile
using Statistics
using LinearAlgebra
using StaticArrays
using StatsPlots
# using ProfileView: @profview
using StatProfilerHTML
using JLD2
using Plots
using ZMQ
using ProtoBuf
using LinearAlgebra
using ForwardDiff
using BlockDiagonals
using ControlSystems
using PyPlot

In [5]:
#include("../plotting.jl")
include("random_linear.jl")
include("random_linear_problem.jl")
#include("/home/fausto/protobuff/juliaout/filtered_state_msg_pb.jl")
#include("/home/fausto/protobuff/juliaout/messaging.jl")

#connect to state publisher 

#contextsub = Context()
#socket1 = Socket(contextsub, SUB)

#println("Collecting state information...")
#connect(socket1, "tcp://localhost:5556") #change to correct one

#subscribe(socket1)

#message = recv(socket)
# message



#Connect to cpp script 
#contextpub = Context()
#socket2 = Socket(contextpub, PUB)
#bind(socket2, "tcp://*:5555")
#iob = IOBuffer()

#functions used in the quaternion math
function hat(v) #skew symmetric matrix operator
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end
function L(q) #the function takes a quaternion as input
    s = q[1] #scalar part of quaternion
    v = q[2:4] #vector part of quaternion
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end

T = Diagonal([1; -ones(3)]) # to find the inverse of a quaternion
H = [zeros(1,3); I]  #zero the scalar part of quaternion
function qtoQ(q) #figure out what this function is for
    return H'*T*L(q)*T*L(q)*H
end

function G(q)
    G = L(q)*H
end

function rptoq(phi)# mapping between rodrigues parameter and quaternion
    (1/sqrt(1+phi'*phi))*[1; phi] #known as the Cayley map
end
function qtorp(q) #mapping between quaternion to rodrigues parameter
    q[2:4]/q[1]
end


qtorp (generic function with 1 method)

In [6]:
# updated quadrotor parameters
m = 1.776 # in kg
l = 0.28 #in m
J = [0.01566089 0.00000318037 0; 0.00000318037 0.01562078 0; 0 0 0.02226868]
g = 9.81
kt=1.1
km=0.44

#umin = [0;0;0;0]
#umax = [0.3*m*g;0.3*m*g;0.3*m*g;0.3*m*g]

h = 0.05 #20 Hz

Nx = 13     #number of states (w quaternion)
Nx_ = 12    #number of states (linearized using rodrigues parameters)
Nu = 4     # number of controls
Tfinal = 5.0 #final time (may change if it is not discrete time)
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));


function E(q)
    E = BlockDiagonal([1.0*I(3), G(q), 1.0*I(6)])
end

function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]
    v = x[8:10]
    omega = x[11:13]
    Q = qtoQ(q)
    
    rdot = Q*v
    qdot = 0.5*L(q)*H*omega
    
    vdot = Q'*[0; 0; -g] + (1/m)*[zeros(2,4); kt*ones(1,4)]*u - hat(omega)*v
    
    omegadot = J\(-hat(omega)*J*omega + [0 l*kt 0 -l*kt; -l*kt 0 l*kt 0; km -km km -km]*u)
    
    return [rdot; qdot; vdot; omegadot]
end

function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] .= xn[4:7]/norm(xn[4:7]) #re-normalize quaternion to reduce error
    return xn #returns the state at a future timestep
end

#Initial Conditions
uhover = (m*g/4)*ones(4) #gravity compensation for a hover
r0 = [0.0; 0; 1.0] #1 meter off the ground
q0 = [1.0; 0; 0; 0]
v0 = zeros(3)
omega0 = zeros(3) #stable hover, no velocities
x0 = [r0; q0; v0; omega0]; #initial/reference state

#Linearize dynamics about hover
A = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),x0)
B = ForwardDiff.jacobian(u->quad_dynamics_rk4(x0,u),uhover);

#reduced system to make it controllable 
A_ = Array(E(q0)'*A*E(q0));
B_ = Array(E(q0)'*B);



In [7]:
function gen_trajectory(n,m,N,dt)
    #prob = gen_random_linear(n,m,N, dt)
    uhover = (m*g/4)*ones(4) #gravity compensation for a hover
    r0 = [0.0; 0; 1.0] #1 meter off the ground
    q0 = [1.0; 0; 0; 0]
    phi0= qtorp(L(q0)'*q0)
    v0 = zeros(3)
    omega0 = zeros(3) #stable hover, no velocities
    x0 = [r0; phi0; v0; omega0]; #initial/reference state
    model = RD.LinearModel(A_,B_)
    #model = LinearizedModel(cmodel, dt=dt, is_affine=true, integration=RD.RK4)
    #discretizedmodel = discretize!(model)
    #print("model discretized")
    
    #initiald = discrete_dynamics(model, x0, uhover)
    #print(initiald)
    #print("done")
    
    # Objective
    Q = Diagonal(10*rand(n))
    R = Diagonal(0.1*ones(m))
    Qf = Q * (N-1)
    x0 = @SVector zeros(n)
    xf = [1; 1; 1; 1; 0; 0; 0; 0; 0; 0; 0; 0]
    obj = LQRObjective(Q, R, Qf, xf, N)
    U = [@SVector randn(m) for k = 1:N-1]
    X = [@SVector zeros(n) for k = 1:N]
    tf = (N-1)*dt

    prob = Problem(model, obj, xf, tf, x0=x0, integration=RD.RK4)
    rollout!(prob)

    for k = 1:N-1
        X[k+1] = discrete_dynamics(TO.integration(prob), prob.model, X[k], U[k], (k-1)*dt, dt)
	#print(X[k])
    end
    Z = Traj(X,U,fill(dt,N-1))
    initial_trajectory!(prob, Z)
    return prob
end

function run_MPC(prob_mpc, opts_mpc, Z_track, num_iters=length(Z_track) - prob_mpc.N)
    # Generate ALTRO solver
    altro = ALTROSolver(prob_mpc, opts_mpc)

    # Generate OSQP solver
    osqp,l,u = gen_OSQP(prob_mpc, opts_mpc)

    # Some initialization
    n,m,N_mpc = size(prob_mpc) 
    xinds = [(k-1)*(n+m) .+ (1:n) for k = 1:N_mpc]
    uinds = [(k-1)*(n+m) + n .+ (1:m) for k = 1:N_mpc-1]
    xi = vcat(xinds...)
    ui = vcat(uinds...)
    x0_l = view(l, (N_mpc-1)*n .+ (1:n))  # views into OSQP constraints
    x0_u = view(u, (N_mpc-1)*n .+ (1:n))
    q = zeros(RD.num_vars(prob_mpc.Z))

    dt = prob_mpc.Z[1].dt
    t0 = prob_mpc.t0
    k_mpc = 1
    obj = prob_mpc.obj

    err_traj = zeros(num_iters,2)
    err_x0 = zeros(num_iters,2)
    iters = zeros(Int, num_iters,2)
    times = zeros(num_iters,2)

    # Solve initial iteration
    solve!(altro)


    return altro 
end

run_MPC (generic function with 2 methods)

In [8]:
opts = SolverOptions(
    cost_tolerance = 1e-4,
    cost_tolerance_intermediate = 1e-4,
    constraint_tolerance = 1e-4,
    penalty_initial = 1_000.,
    penalty_scaling = 100.,
    reset_duals = false,
    projected_newton = false
)


SolverOptions{Float64}
  constraint_tolerance: Float64 0.0001
  cost_tolerance: Float64 0.0001
  cost_tolerance_intermediate: Float64 0.0001
  gradient_tolerance: Float64 10.0
  gradient_tolerance_intermediate: Float64 1.0
  iterations_inner: Int64 300
  dJ_counter_limit: Int64 10
  square_root: Bool false
  line_search_lower_bound: Float64 1.0e-8
  line_search_upper_bound: Float64 10.0
  iterations_linesearch: Int64 20
  max_cost_value: Float64 1.0e8
  max_state_value: Float64 1.0e8
  max_control_value: Float64 1.0e8
  static_bp: Bool true
  save_S: Bool false
  bp_reg: Bool false
  bp_reg_initial: Float64 0.0
  bp_reg_increase_factor: Float64 1.6
  bp_reg_max: Float64 1.0e8
  bp_reg_min: Float64 1.0e-8
  bp_reg_type: Symbol control
  bp_reg_fp: Float64 10.0
  penalty_initial: Float64 1000.0
  penalty_scaling: Float64 100.0
  active_set_tolerance_al: Float64 0.001
  dual_max: Float64 1.0e8
  penalty_max: Float64 1.0e8
  iterations_outer: Int64 30
  kickout_max_penalty: Bool false
  re

In [9]:
#Random.seed!(1)
N = 1101
Ns = [11,31,51,71,101]
n = 12
m = 4
dt = 0.05

prob = gen_trajectory(n, m, N, dt)
Z_track = prob.Z
#results = map(Ns) do N_mpc
println("Running with 101 knot points...")
prob_mpc = gen_tracking_problem(prob, 101)
solver = run_MPC(prob_mpc, opts, Z_track, 100) 
quadcontrols = controls(solver)
print(quadcontrols[1])
#end
#@show [median(res[:iter], dims=1) for res in results]
#@show [median(res[:time], dims=1) for res in results]
#@save "horizon_comp.jld2" results Ns

#Simulate quad
uhist = zeros(Nu,Nt)
xhist = zeros(Nx,Nt)
xhist[:,1] = [r0+randn(3); L(q0)*rptoq(0.7*randn(3)); v0; omega0]
for k = 1:(Nt-1)
    uhist[:,k] = quadcontrols[k]
    xhist[:,k+1] = quad_dynamics_rk4(xhist[:,k],uhist[:,k])
    #print(uhist[:,k])
end

Running with 101 knot points...

SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 1
    Solve Time: 1.948315 (ms)

  Covergence
    Terminal Cost: 3.088652814411952e-13
    Terminal dJ: 0.0
    Terminal gradient: 0.0
    Solve Status: SOLVE_SUCCEEDED
[0.17155545155385918, 0.7101805475387761, 1.8375290542396818, 0.3099471617636368]

In [11]:
plot(thist,xhist[1,:], label="x LQR")
xlabel("time")
legend()
show()

LoadError: UndefVarError: plot not defined

In [12]:
using TrajOptPlots
using MeshCat
using RobotZoo

vis = Visualizer()
TrajOptPlots.set_mesh!(vis, RobotZoo.Quadrotor())
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/fausto/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8700))

In [13]:
X1 = [SVector{13}(x) for x in eachcol(xhist)];
visualize!(vis, RobotZoo.Quadrotor(), thist[end], X1)